In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
import selenium
import requests

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
driver = webdriver.Chrome('/Users/dinhkytran/Downloads/Webscrapping/chromedriver')

/var/folders/tc/6yb9xrw54650c37qtb_96twc0000gn/T/ipykernel_24000/2957874724.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/dinhkytran/Downloads/Webscrapping/chromedriver')


In [4]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import pandas as pd

In [10]:
def get_jobs(verbose, path, num_jobs):

    '''Gathers jobs as a dataframe, scraped from Glassdoor'''

    #Initializing the webdriver
    options = webdriver.ChromeOptions()

    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')

    driver = webdriver.Chrome(executable_path="/Users/dinhkytran/Downloads/Webscrapping/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    url = "https://www.glassdoor.de/Job/supply-chain-jobs-SRCH_KO0,12.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&typedLocation=&context=Jobs"
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        #time.sleep(slp_time)

       

        #Going through each job in the current page
        job_buttons = driver.find_elements(By.CLASS_NAME,"jobLink job-search-key-1rd3saf eigr9kq1")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
            try:
                job_button.click()  #You might 
                time.sleep(1)
                collected_successfully = False
            except:
                continue
            while not collected_successfully:
                try:
                    company_name = driver.find_element("xpath",'.//div[@class="d-flex justify-content-between align-items-start"]').text
                    location = driver.find_element("xpath",'.//div[@class="d-flex flex-wrap job-search-key-1m2z0go e1rrn5ka2"]').text
                    job_title = driver.find_element("xpath",'.//div[@class="d-flex flex-column pl-sm css-1buaf54 job-search-key-1mn3dn8 e1rrn5ka0"]').text
                    #job_description = driver.find_element("xpath",'//*[@id="JobDesc1007958880704"]/div/div/div[1]/div[3]/h3').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element("xpath",'.//div[@class="css-y2jiyn e2u4hf18"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."

            try:
                rating = driver.find_element("xpath",'//*[@id="JDCol"]/div/article/div/div[2]/div[1]/div[2]/div/div[1]/div[2]/div[1]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

     #Clicking on the "next page" button
        try:
            page = driver.find_element("xpath",'.//button[@class="nextButton css-1hq9k8 e13qs2071"]').text
            page = page.split()
            if page[1]==page[3]:
                break
            driver.find_element("xpath",'.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
    return pd.DataFrame(jobs).reset_index()  #This line converts the dictionary object into a pandas DataFrame.


In [9]:
path = "chromedriver"
df = get_jobs(False, path, 400)

/var/folders/tc/6yb9xrw54650c37qtb_96twc0000gn/T/ipykernel_24000/3850156546.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/Users/dinhkytran/Downloads/Webscrapping/chromedriver", options=options)


Scraping terminated before reaching target number of jobs. Needed 400, got 0.
